In [1]:
import numpy as np
import random

In [2]:
class Car:
    def __init__(self, x_0, v_0):
        self.x = x_0
        self.v = v_0
        self.has_encounter = False

    def normal_move(self, step):
        self.x = self.x + step * self.v

    def velocity(self):
        return self.x

class Road:
    def __init__(self, a, z, N, T):
        self.cars = self.generate_cars(z, N, T)
        self.a = a
        self.z = z
        self.N = N
        self.T = T
    
    def generate_cars(self, z, N, T):
        num_cars = int(np.random.poisson(z * T))
        return [Car(x_0 = random.randint(0, N), v_0 = random.randint(1, 2)) for _ in range(num_cars)]

    def run_one_sim(self):
        miles_not_driven = 0
        for t in range(self.T):
            for car in self.cars:
                car.normal_move(step=1)

            positions = {}
            for car in self.cars:
                pos = car.x
                if pos in positions:
                    positions[pos].append(car)
                else:
                    positions[pos] = [car]
            
            for colliding_cars in positions.values():
                if len(colliding_cars) != 2:
                    continue
                car1 = colliding_cars[0]
                car2 = colliding_cars[1]
                miles_not_driven += self.miles_not_driven(car1.v, car2.v)
                car1.has_encounter = True
                car2.has_encounter = True
        return miles_not_driven
                
    def miles_not_driven(self, c1, c2):
        slow_car = min(c1, c2)
        if (c1 > self.a and c2 <= self.a) or (c2 > self.a and c1 <= self.a):
            return 0
        if c1 <= self.a and c2 <= self.a:
            decel_time_lost = slow_car 
            s = (slow_car * decel_time_lost - 0.5 * decel_time_lost**2) + (0 * decel_time_lost  + 0.5 * decel_time_lost**2)  
            s_0 = slow_car * 2 * decel_time_lost

        else:
            delta_v = slow_car - self.a
            t = delta_v  
            s = slow_car * t - 0.5 * t**2 + self.a * t + 0.5 * t**2 
            s_0 = slow_car * 2 * t 
        return s_0 - s
    
    def run_sims(self, nsims):
        total_miles_not_driven = 0
        for _ in range(nsims):
            miles_not_driven_sim = self.run_one_sim()
            total_miles_not_driven += miles_not_driven_sim
        return total_miles_not_driven / nsims

In [4]:
a = np.linspace(1, 2, 100)
z = 0.001
N = 10000
T = 10000
nsims = 100
results = {}
for a_i in a:
    road_ai = Road(a = a_i, z=z, N=N, T=T)
    miles_lost_ai = road_ai.run_sims(nsims=nsims)
    results[a_i] = miles_lost_ai
    print(f'{a_i} value finished with {miles_lost_ai} miles lost')

1.0 value finished with 0.0 miles lost
1.0101010101010102 value finished with 0.0 miles lost
1.02020202020202 value finished with 0.0 miles lost


KeyboardInterrupt: 